In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
modelname="NASNetMobile_dropflatten_512"
LR=0.001
DO=0.2

In [3]:
foldernames_to_id={"VeryMildDemented":1, "NonDemented":0, "ModerateDemented":2, "MildDemented":3}
import os
import torch

In [4]:
from PIL import Image
from numpy import asarray
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers,models
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, Flatten, MaxPooling2D, Dense, Dropout, GlobalAveragePooling2D
from keras import optimizers, losses
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
import os.path
from sklearn.metrics import classification_report, confusion_matrix
import itertools

In [5]:
train_datagen = ImageDataGenerator(rescale=1./255,
    validation_split=0.1) # set validation split

In [6]:
foldername='/content/drive/My Drive/ml_project/Dataset/'
train_images = train_datagen.flow_from_directory(
    foldername,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training') # set as training data

validation_images = train_datagen.flow_from_directory(
    foldername , # same directory as training data
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation') # set as validation data

Found 30586 images belonging to 4 classes.
Found 3398 images belonging to 4 classes.


In [7]:
def create_model():
  mymodel = Sequential()

  pretrained_model= tf.keras.applications.NASNetMobile(include_top=False,
                   input_shape=(224,224,3),
                   pooling='avg',classes=4,
                   weights='imagenet')
  for layer in pretrained_model.layers:
        layer.trainable=False

  mymodel.add(pretrained_model)

  mymodel.add(Flatten())
  mymodel.add(Dropout(DO))
  mymodel.add(Dense(512, activation='relu'))
  mymodel.add(Dropout(DO))
  mymodel.add(Dense(4, activation='softmax'))
  mymodel.compile(loss = 'categorical_crossentropy', optimizer = tf.keras.optimizers.Adam(lr=LR), metrics = ['accuracy'])
  return mymodel

In [8]:
# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 NASNet (Functional)         (None, 1056)              4269716   
                                                                 
 flatten (Flatten)           (None, 1056)              0         
                                                                 
 dropout (Dropout)           (None, 1056)              0         
                                                                 
 dense (Dense)               (None, 512)               541184    
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 4)                 2052      
                                                                 
Total params: 4,812,952
Trainable params: 543,236
Non-tr

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [9]:
# Include the epoch in the file name (uses `str.format`)
checkpointdir='/content/drive/My Drive/ml_project/Checkpoints/'
checkpoint_path = checkpointdir+modelname+"training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 32

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=20*batch_size)

# Create a new model instance
model = create_model()

# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

In [10]:
checkpoint_dir

'/content/drive/My Drive/ml_project/Checkpoints/NASNetMobile_dropflatten_512training_1'

In [11]:
# Loads the weights
#p = checkpointdir+modelname+"-training/cp-"
p= checkpointdir+modelname
latest = tf.train.latest_checkpoint(checkpoint_dir)
print(latest)
latest="/content/drive/My Drive/ml_project/Checkpoints/NASNetMobile_dropflatten_512-training/cp-0071.ckpt"
model.load_weights(latest)

/content/drive/My Drive/ml_project/Checkpoints/NASNetMobile_dropflatten_512training_1/cp.ckpt


In [12]:
# Evaluate the model
#loss, acc = model.evaluate(validation_images, verbose=2)
#print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))

start from epch 55+71

In [ ]:
history = model.fit(train_images,
    steps_per_epoch=len(train_images),
    validation_data=validation_images,
    validation_steps=len(validation_images),
    epochs=300,
    callbacks=[cp_callback])

Epoch 1/300
639/956 [===================>..........] - ETA: 28s - loss: 0.3400 - accuracy: 0.8608
Epoch 1: saving model to /content/drive/My Drive/ml_project/Checkpoints/NASNetMobile_dropflatten_512training_1/cp.ckpt
956/956 [==============================] - 111s 102ms/step - loss: 0.3372 - accuracy: 0.8622 - val_loss: 0.3511 - val_accuracy: 0.8534
Epoch 2/300
323/956 [=========>....................] - ETA: 54s - loss: 0.3321 - accuracy: 0.8633
Epoch 2: saving model to /content/drive/My Drive/ml_project/Checkpoints/NASNetMobile_dropflatten_512training_1/cp.ckpt
956/956 [==============================] - 90s 94ms/step - loss: 0.3406 - accuracy: 0.8604 - val_loss: 0.3547 - val_accuracy: 0.8567
Epoch 3/300
  7/956 [..............................] - ETA: 1:17 - loss: 0.3700 - accuracy: 0.8440
Epoch 3: saving model to /content/drive/My Drive/ml_project/Checkpoints/NASNetMobile_dropflatten_512training_1/cp.ckpt
647/956 [===================>..........] - ETA: 27s - loss: 0.3354 - accuracy: 0

In [ ]:
#Accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
# loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

2 following cells are for loading latest trained models

In [ ]:
# Create a basic model instance
#model = create_model()

# Evaluate the model
#loss, acc = model.evaluate(validation_images, verbose=2)
#print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))

In [ ]:
# Loads the weights
#p = checkpointdir+modelname+"-training/cp-"

#fill in the best model filename like 0004
#best_checkpoint_path=p+""+"".ckpt"
#latest = tf.train.latest_checkpoint(checkpoint_dir)
#model.load_weights(latest)

# Re-evaluate the model
#loss, acc = model.evaluate(validation_images, verbose=2)
#print("Restored model, accuracy: {:5.2f}%".format(100 * acc))